<a href="https://colab.research.google.com/github/TariqAlhathloul/Smart-Street/blob/main/WorkSpace/train_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## linkes for the project

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow.svg)](https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3)

[![GitHub Repoistory](https://badges.aleen42.com/src/github.svg)](https://github.com/TariqAlhathloul/Smart-Street)


## installation

In [ ]:
!pip install ultralytics -q
!pip install roboflow -q
!pip install ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd
import os

from google.colab import userdata
API_KEY = userdata.get('ROBOFLOW_API_KEY')

In [ ]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-3")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.103, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Road-Lines-Segmentation-3-2 in yolov8:: 100%|██████████| 35614/35614 [00:04<00:00, 7827.97it/s]


## upload the dataset

In [ ]:
TRAIN_DIR_IAMAGES = '/content/Road-Lines-Segmentation-3-2/train/images'
VAL_DIR_IAMGES = '/content/Road-Lines-Segmentation-3-2/valid/images'
TEST_DIR_IMAGES = '/content/Road-Lines-Segmentation-3-2/test/images'

In [ ]:
print(f"Total Training: {len(os.listdir(TRAIN_DIR_IAMAGES))} images")
print(f"Total Validation: {len(os.listdir(VAL_DIR_IAMGES))} images")
print(f"Total Testing: {len(os.listdir(TEST_DIR_IMAGES))} images")

Total Training: 14979 images
Total Validation: 1816 images
Total Testing: 1006 images


## check if the GPU is avalabie

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 29 15:25:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## upload the model

In [ ]:
#model = YOLO('/kaggle/input/best/pytorch/default/1/best.pt')
# first experiment using the pervious model from the mini project

In [ ]:
# second experiment usning yolov8n-seg
model = YOLO("yolov8n-seg.pt")

100%|██████████| 6.74M/6.74M [00:00<00:00, 24.2MB/s]


## overwrite the data.yaml file

In [ ]:
%%writefile /content/Road-Lines-Segmentation-3-2/data.yaml
names:
- car
- license_plate
- solid-yellow-line
nc: 3
roboflow:
  license: CC BY 4.0
  project: road-lines-segmentation-3
  url: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3/dataset/2
  version: 2
  workspace: capstoneteam
test: /content/Road-Lines-Segmentation-3-2/test/images
train: /content/Road-Lines-Segmentation-3-2/train/images
val: /content/Road-Lines-Segmentation-3-2/valid/images


Overwriting /content/Road-Lines-Segmentation-3-2/data.yaml


## Hyperparameters, finetuning

In [ ]:
IMG_SIZE = 640
BATCH_SIZE = -1
EPOCHS = 100

In [ ]:
results = model.train(
    data = '/content/Road-Lines-Segmentation-3-2/data.yaml',
    task = 'segment',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Segment-Model-V5',
    project = 'Model-V5',
    patience = 20,
    save = True,
    optimizer = 'SGD',
    lr0 = 0.001,
    lrf = 0.0001,
    momentum = 0.937,
    weight_decay = 0.0005,
    cos_lr=True,
    profile=True,
    freeze=10,
    dropout=0.2
)

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/Road-Lines-Segmentation-3-2/data.yaml, epochs=100, time=None, patience=20, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Model-V5, name=Segment-Model-V5, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=93, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=True, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

100%|██████████| 755k/755k [00:00<00:00, 4.28MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 22.8MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.08G reserved, 0.07G allocated, 14.60G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     3264201       12.11         0.218         36.27           nan        (1, 3, 640, 640)                    list
     3264201       24.22         0.352         21.23           nan        (2, 3, 640, 640)                    list
     3264201       48.45         0.640         21.23           nan        (4, 3, 640, 640)                    list
     3264201       96.89         1.214         36.53           nan        (8, 3, 640, 640)                    list
     3264201       193.8         2.294         50.13           nan       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 62 for CUDA:0 8.84G/14.75G (60%) ✅


...

     60/100      6.45G      0.546     0.7971     0.4331     0.8377        128        640: 100%|██████████| 242/242 [02:32<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:16<00:00,  1.09s/it]

                   all       1816       7060      0.754      0.726      0.759      0.601       0.71      0.683      0.701       0.46
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 40, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



60 epochs completed in 2.808 hours.
Optimizer stripped from Model-V5/Segment-Model-V5/weights/last.pt, 6.8MB
Optimizer stripped from Model-V5/Segment-Model-V5/weights/best.pt, 6.8MB

Validating Model-V5/Segment-Model-V5/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,649 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:18<00:00,  1.21s/it]


                   all       1816       7060      0.756      0.719      0.763      0.599      0.732      0.684      0.714      0.463
                   car       1661       4890      0.868      0.864      0.925      0.779      0.876      0.848      0.918      0.668
         license_plate        181        269      0.615      0.349      0.419      0.196      0.521      0.268      0.279       0.08
     solid-yellow-line       1746       1901      0.785      0.943      0.945      0.822      0.798      0.935      0.944       0.64
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to Model-V5/Segment-Model-V5


## evaluation on the validationset


In [ ]:
evaluation = model.val(
    data = '/content/Road-Lines-Segmentation-3-2/data.yaml',
    task = 'segment',
    imgsz = 640,
    iou = 0.6,
    batch = 32,
    save_json = True,
    save_txt = True,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-evaluation-5'
)

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/Road-Lines-Segmentation-3-2/valid/labels.cache... 1816 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1816/1816 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/Road-Lines-Segmentation-3-2/valid/images/front-left-NO20240903-145041-000844F_MP4-0244_jpg.rf.d35bf3f6a9afb53703cdd639f4526b98.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [02:06<00:00,  2.21s/it]


                   all       1816       7060      0.739       0.74      0.767      0.601      0.717      0.725      0.738      0.444
                   car       1661       4890      0.851      0.884      0.927      0.784      0.851      0.883      0.926      0.683
         license_plate        181        269      0.587      0.392       0.43      0.202       0.52      0.347      0.346      0.108
     solid-yellow-line       1746       1901       0.78      0.944      0.946      0.818      0.781      0.945      0.942      0.541
Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Saving Model-V5/Model-segment-evaluation-53/predictions.json...
Results saved to Model-V5/Model-segment-evaluation-53


## predictions on the testset

In [ ]:
predictions = model.predict(
    data = '/content/Road-Lines-Segmentation-3-2/data.yaml',
    source = '/content/Road-Lines-Segmentation-3-2/test/images',
    task = 'segment',
    imgsz = 640,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-predictions',
    save_json = True,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1006 /kaggle/working/Road-Lines-Segmentation-3-2/test/images/NO20240828-152333-000152F_MP4-0008_jpg.rf.887ba244a440edadb93ef7968520f916.jpg: 640x640 8 cars, 10.6ms
image 2/1006 /kaggle/working/Road-Lines-Segmentation-3-2/test/images/NO20240828-152333-000152F_MP4-0075_jpg.rf.7773aeaf1934897b09a365966051ed5b.jpg: 640x640 4 cars, 7.3ms
image 3/1006 /kaggle/working/Road-Lines-Segmentation-3-2/test/images/NO20240828-152333-000152F_MP4-0078_jpg.rf.cb

## apply quantization

In [ ]:
model.export(format="onnx", int8=True, dynamic=True,half=True, simplify=True, data='/content/Road-Lines-Segmentation-3-2/data.yaml')

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
WARNING ⚠️ half=True and int8=True are mutually exclusive, setting half=False.

PyTorch: starting from 'Model-V5/Segment-Model-V5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 39, 8400), (1, 32, 160, 160)) (6.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim==0.1.34', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 249.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 266.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 245.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 324.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 13.1s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim==0.1.34', 'onnxruntime-gpu']
requirement

'Model-V5/Segment-Model-V5/weights/best.onnx'

In [ ]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-3")
project.version(2).deploy(model_type="yolov8n", model_path="/content/Model-V5/Segment-Model-V5")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.103, to fix: `pip install ultralytics==8.0.196`
Would you like to continue with the wrong version of ultralytics? y/n: y
View the status of your deployment at: https://app.roboflow.com/capstoneteam/road-lines-segmentation-3/2
Share your model with the world at: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3/model/2


In [ ]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!zip -r model.zip /content/Model-V5/Segment-Model-V5/

updating: content/Model-V5/Segment-Model-V5/ (stored 0%)
  adding: content/Model-V5/Segment-Model-V5/val_batch0_pred.jpg (deflated 8%)
  adding: content/Model-V5/Segment-Model-V5/BoxPR_curve.png (deflated 10%)
  adding: content/Model-V5/Segment-Model-V5/train_batch0.jpg (deflated 4%)
  adding: content/Model-V5/Segment-Model-V5/results.csv (deflated 85%)
  adding: content/Model-V5/Segment-Model-V5/results.png (deflated 9%)
  adding: content/Model-V5/Segment-Model-V5/confusion_matrix.png (deflated 28%)
  adding: content/Model-V5/Segment-Model-V5/MaskF1_curve.png (deflated 9%)
  adding: content/Model-V5/Segment-Model-V5/MaskR_curve.png (deflated 9%)
  adding: content/Model-V5/Segment-Model-V5/MaskP_curve.png (deflated 9%)
  adding: content/Model-V5/Segment-Model-V5/roboflow_deploy.zip (stored 0%)
  adding: content/Model-V5/Segment-Model-V5/events.out.tfevents.1727623941.06805434b498.712.0 (deflated 89%)
  adding: content/Model-V5/Segment-Model-V5/state_dict.pt (deflated 9%)
  adding: cont

In [ ]:
!zip -r evaluation.zip /content/Model-V5/Model-segment-evaluation-53

updating: content/Model-V5/Model-segment-evaluation-53/ (stored 0%)
  adding: content/Model-V5/Model-segment-evaluation-53/val_batch0_pred.jpg (deflated 9%)
  adding: content/Model-V5/Model-segment-evaluation-53/BoxPR_curve.png (deflated 10%)
  adding: content/Model-V5/Model-segment-evaluation-53/confusion_matrix.png (deflated 28%)
  adding: content/Model-V5/Model-segment-evaluation-53/MaskF1_curve.png (deflated 9%)
  adding: content/Model-V5/Model-segment-evaluation-53/MaskR_curve.png (deflated 9%)
  adding: content/Model-V5/Model-segment-evaluation-53/MaskP_curve.png (deflated 10%)
  adding: content/Model-V5/Model-segment-evaluation-53/BoxP_curve.png (deflated 11%)
  adding: content/Model-V5/Model-segment-evaluation-53/confusion_matrix_normalized.png (deflated 28%)
  adding: content/Model-V5/Model-segment-evaluation-53/MaskPR_curve.png (deflated 11%)
  adding: content/Model-V5/Model-segment-evaluation-53/val_batch1_labels.jpg (deflated 8%)
  adding: content/Model-V5/Model-segment-eva